In [5]:
import findspark
findspark.init('/home/mj1e16/miniconda2/lib/python2.7/site-packages/pyspark')

In [6]:
from astropy.io import fits

In [7]:
#import astroprov
import numpy as np
import os
import random

from astropy.visualization.mpl_normalize import ImageNormalize
from astropy.table import Table
from astropy.table import Column

import collections
import subprocess

import time
from multiprocessing import Pool
from multiprocessing.pool import ThreadPool

from functools import partial

from astropy.io import ascii
#from sqlalchemy import create_engine
import sqlite3
import mysql.connector
from pandas import DataFrame

import mysql.connector
from mysql.connector import Error
from mysql.connector import pooling

from pyspark.sql import Row
from pyspark.sql import SQLContext
from pyspark import SparkContext
from pyspark import SparkConf
import datetime
from sqlalchemy import create_engine
import sys


In [91]:
defaultDirectory = '/home/mj1e16/sextractor/sextractor-master/config/'

In [68]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /home/mj1e16/miniconda2/lib/python2.7/site-packages/pyspark/mysql-connector-java_8.0.16-1ubuntu16.04_all/usr/share/java/mysql-connector-java-8.0.16.jar  pyspark-shell'
config = SparkConf().setAll([('spark.executor.cores', '6'),('spark.cores.max', '6'),('spark.driver.memory','1g'),('spark.executor.memory', '500m'),("spark.sql.execution.arrow.enabled", "true")]) # ("spark.sql.execution.arrow.enabled", "true")


In [69]:
sc = SparkContext(appName='App',conf=config)
sqlContext = SQLContext(sc)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=App, master=local[*]) created by __init__ at <ipython-input-9-f19d620c11a2>:1 

In [10]:

mags = [7,6,5,4,3,2,1]
for mag in mags:
    dataframe_mysql = sqlContext.read.format("jdbc").option("url", "jdbc:mysql://localhost/Kepler").option("driver", "com.mysql.jdbc.Driver").option("dbtable", "starlist_{}".format(mag)).option("user", "mj1e16").option("password", "[sqlT1G3R]").load()
    dataframe_mysql.registerTempTable('starlist_{}'.format(mag))

In [92]:
def innerJoin(tableName,totGross,totGrossOriginal,variableList,mag,baseNum,diffSize=1,xlength=1015.,ylength=1085.,weight=0.5):
    nsegs = 4
    
    #mag = variableList[6]
    temptableName = tableName[0:5]+tableName[-1]
    OGtemptableName = 'original'
    OGdf = sqlContext.sql("SELECT NUMBER, MAG_BEST, X_IMAGE, Y_IMAGE FROM {0} INNER JOIN starlist_{1} ON {0}.X_IMAGE BETWEEN starlist_{1}.X_POS_MIN AND starlist_{1}.X_POS_MAX AND {0}.Y_IMAGE BETWEEN starlist_{1}.Y_POS_MIN AND starlist_{1}.Y_POS_MAX".format(OGtemptableName,abs(mag)))
    OGdf.registerTempTable(OGtemptableName+'match')
    sqlContext.cacheTable(OGtemptableName+'match')
    OGtotNumber = OGdf.count()
    
    df = sqlContext.sql("SELECT NUMBER, MAG_BEST, X_IMAGE, Y_IMAGE FROM {0} INNER JOIN starlist_{1} ON {0}.X_IMAGE BETWEEN starlist_{1}.X_POS_MIN AND starlist_{1}.X_POS_MAX AND {0}.Y_IMAGE BETWEEN starlist_{1}.Y_POS_MIN AND starlist_{1}.Y_POS_MAX".format(temptableName,abs(mag)))
    df.registerTempTable(temptableName+'match')
    sqlContext.cacheTable(temptableName+'match')
    totNumber = df.count()
    #print('tot',totNumber)
    
    xsegment = xlength/nsegs
    ysegment = ylength/nsegs
    
    accuracyScore = 1 - (OGtotNumber/baseNum)
    completenessScore = 0
    #totSeg = []
    #df.write.format('jdbc').options(url='jdbc:mysql://localhost/Kepler',driver='com.mysql.jdbc.Driver',dbtable='result_{}'.format(tableName),user='mj1e16',password='[sqlT1G3R]').mode('append').save()
    for xsegs in range(nsegs):
        xmin = int(xsegs*xsegment)+15
        xmax = int(xmin+xsegment)+15
        for ysegs in range(nsegs):
            ymin = int(ysegs*ysegment)+15
            ymax = int(ymin+ysegment)+15
            
            OGdf = sqlContext.sql("SELECT NUMBER FROM {0} WHERE {0}.X_IMAGE BETWEEN {1} AND {2} AND {0}.Y_IMAGE BETWEEN {3} AND {4}".format(OGtemptableName+'match',xmin,xmax,ymin,ymax))
            OGnumber = OGdf.count()
            df = sqlContext.sql("SELECT NUMBER FROM {0} WHERE {0}.X_IMAGE BETWEEN {1} AND {2} AND {0}.Y_IMAGE BETWEEN {3} AND {4}".format(temptableName+'match',xmin,xmax,ymin,ymax))
            number = df.count()
            simulated = number-OGnumber
            
            completenessScore +=  62 - simulated #abs(simulated-62)
            if completenessScore < 0:
                completenessScore = 0 
            variableList.append(simulated)
            
#             dftot = sqlContext.sql("SELECT NUMBER FROM {0} WHERE {0}.X_IMAGE BETWEEN {1} AND {2} AND {0}.Y_IMAGE BETWEEN {3} AND {4}".format(temptableName,xmin,xmax,ymin,ymax))
#             number = df.count()
#             #variableList.append(number)
#             totSeg.append(number)

            # need to create some place to write to

    
    
#    variableList.extend(totSeg)
    #astroprov.provcall([tableName,'starlist_{}'.format(abs(mag))],['result_{}'.format(tableName)],"innerJoin_Python2Python_SQ_tmpl.provn","innerJoin",provDir)
    compScore = 1 - (completenessScore/992)
    score = (weight*completenessScore) + ((1-weight)*accuracyScore)
    
    sqlContext.uncacheTable(temptableName+'match')
    sqlContext.dropTempTable(temptableName+'match')
    sqlContext.uncacheTable(OGtemptableName+'match')
    sqlContext.dropTempTable(OGtemptableName+'match')
    sqlContext.uncacheTable(temptableName)
    sqlContext.dropTempTable(temptableName)
    sqlContext.uncacheTable(OGtemptableName)
    sqlContext.dropTempTable(OGtemptableName)
    variableList.append(totNumber)
    variableList.append(totGross)
    variableList.append(OGtotNumber)
    variableList.append(totGrossOriginal) 
    variableList.append(score)
    variableTuple = tuple(variableList)
    #print(variableTuple)
    return [variableTuple,score]


def makeConfig(valList,tableName='table',defaultDir=defaultDirectory,attributeList=['DETECT_THRESH','DETECT_MINAREA','FILTER_NAME']):
    
    with open(defaultDir+'/default.sex','r') as f:
        data  = f.read()
    for x in range(len(valList)):
        nameLoc = data.find(attributeList[x]) + len(attributeList[x])
        endLoc = data[nameLoc:].find('#') + nameLoc
        newData = data[:nameLoc] + ' '+str(valList[x])+' ' + data[endLoc:]
        data = newData
    
    cname = 'CATALOG_NAME'
    catName = 'test1.cat'
    confName = 'default_1.sex'
    nameLoc = data.find(cname) + len(cname)
    endLoc = data[nameLoc:].find('#') + nameLoc
    newData = data[:nameLoc] + ' ' +catName+ ' ' + data[endLoc:]
    data = newData

    cname = 'PARAMETERS_NAME'
    nameLoc = data.find(cname) + len(cname)
    endLoc = data[nameLoc:].find('#') + nameLoc
    newData = data[:nameLoc] + ' autodefault.param ' + data[endLoc:]
    
    #print(newData)
    with open(defaultDir+confName,'w') as f:
        f.write(newData)
    return(confName,tableName,catName)

def findObjects(confName,tableName,catName,defaultDir=defaultDirectory,imagename='/home/mj1e16/iraf/editedImage5000.fits',original='no',base='no'):
    
    os.chdir(defaultDir)
    subprocess.call(['sex',imagename,'-c',confName])
    assoc = Table.read(catName,format='ascii.sextractor')
    df = assoc.to_pandas()
    df2 = df[df['MAG_BEST'] != 99.0000]
    df_spark = sqlContext.createDataFrame(df2)
    temptableName = tableName[0:5]+tableName[-1]
    if original == 'yes':
        temptableName = 'original'
    if base == 'yes':
        temptableName = 'baseSettings'
    df_spark.registerTempTable(temptableName)
    totGross = df_spark.count()
    sqlContext.cacheTable(temptableName)

    return totGross

def alltogethernow(valList,tableName,IMAGE,variableList,originalImage,mag,atList,baseNum):
    Names = makeConfig(valList,tableName=tableName,attributeList=atList)
    totGrossOriginal = findObjects(Names[0],Names[1],Names[2],imagename=originalImage,original='yes')
    totGross = findObjects(Names[0],Names[1],Names[2],imagename=IMAGE) # confName tabName catname
    finalTuple = innerJoin(Names[1],totGross,totGrossOriginal,variableList,mag,baseNum)
    return finalTuple # named final tuple but actually [finaltuple,score]

In [12]:
def hotpantsQuality(image):
    hdu = fits.open(image)
    imData = hdu[0].data
    imData = abs(imData)
    totVal = sum(imData)
    totVal = sum(totVal)
    return totVal

In [13]:
valList = [np.linspace(1,10,10),np.linspace(1,9,9),['default.conv','gauss_1.5_3x3.conv','gauss_2.0_3x3.conv','gauss_2.0_5x5.conv',
                                                    'gauss_2.5_5x5.conv','gauss_3.0_5x5.conv','gauss_3.0_7x7.conv',
                                                    'gauss_4.0_7x7.conv','gauss_5.0_9x9.conv','mexhat_1.5_5x5.conv',
                                                    'mexhat_2.0_7x7.conv','mexhat_2.5_7x7.conv','mexhat_3.0_9x9.conv',
                                                    'mexhat_4.0_9x9.conv','mexhat_5.0_11x11.conv','tophat_1.5_3x3.conv',
                                                    'tophat_2.0_3x3.conv','tophat_2.5_3x3.conv','tophat_3.0_3x3.conv',
                                                    'tophat_4.0_5x5.conv','tophat_5.0_5x5.conv']]

In [71]:
def evaluateImage(valList,simImage,OGImage,ccd,median,norm,minmag,smallName='newRun',atList=['DETECT_THRESH','DETECT_MINAREA','FILTER_NAME']):
    
    finalTableTuples = []
    t0 = time.time()

    for x0 in range(len(valList[0])):
        for x1 in range(len(valList[1])):
                name = smallName+'_'+str(x0)+'_'+str(x1)+'_'
                for x2 in range(len(valList[2])):
                    variableList = [float(valList[0][x0]),float(valList[1][x1]),valList[2][x2],ccd,median,minmag,norm]
                    fullname = name+str(x2)
                    fullValList = [valList[0][x0],valList[1][x1],valList[2][x2]]
                    print(fullname)
                    try:
                        alltogethernowResults = alltogethernow(fullValList,tableName=fullname,IMAGE=simImage,variableList=variableList,originalImage=OGImage,mag=minmag,atList=atList)
                        finalTableTuples.append(alltogethernowResults[0])
                        score = alltogethernowResults[1]
                    except RuntimeError:
                        with open('redos','a') as f:
                            bigString = simImage+','+str(fullname)+','+str(fullValList)
                            f.write(bigString)
                        print('Redo with valList = ',fullValList)
                        pass

    print(finalTableTuples)
    rdd = sc.parallelize(finalTableTuples)
    kepler = rdd.map(lambda x: Row(detectThresh=x[0],detectMinarea=x[1],filterName=x[2],ccd=x[3],median=x[4],medianQuality=x[5],
                                   minmag=x[6], xy_0=int(x[7]),xy_1=int(x[8]),xy_2=int(x[9]),xy_3=int(x[10]),xy_4=int(x[11]),
                                   xy_5=int(x[12]),xy_6=int(x[13]),xy_7=int(x[14]),xy_8=int(x[15]),xy_9=int(x[16]),xy_10=int(x[17]),
                                   xy_11=int(x[18]),xy_12=int(x[19]),xy_13=int(x[20]),xy_14=int(x[21]),xy_15=int(x[22]),
                                   totNum=int(x[23]),totGross=int(x[24]),OGtot=int(x[25]),OGtotGross=int(x[26]),score=int(x[27])))

    schemaKepler = sqlContext.createDataFrame(kepler)
    #print('finalResults_{}_{}_{}'.format(ccd,median,minmag))
    schemaKepler.write.format('jdbc').options(url='jdbc:mysql://localhost/Kepler',driver='com.mysql.jdbc.Driver',dbtable='finalResultsNewRun_{}_{}_{}'.format(ccd,median,abs(minmag)),user='mj1e16',password='[sqlT1G3R]').mode('append').save()
    
    print(time.time()-t0)


In [59]:
def oneSettingEvaluate(valList,simImage,OGImage,ccd,median,norm,minmag,baseNum,smallName='newRun',starting=[0,0,0],atList=['DETECT_THRESH','DETECT_MINAREA','FILTER_NAME']):
    
    fullname = smallName+'_'+str(starting[0])+'_'+str(starting[1])+'_'+str(starting[2])               
    variableList = [float(valList[0][starting[0]]),float(valList[1][starting[1]]),valList[2][starting[2]],ccd,median,minmag,norm]
    fullValList = [valList[0][starting[0]],valList[1][starting[1]],valList[2][starting[2]]]
    print(fullname)
    print(starting)
    try:
        alltogethernowResults = alltogethernow(fullValList,tableName=fullname,IMAGE=simImage,variableList=variableList,originalImage=OGImage,mag=minmag,atList=atList,baseNum)
        finalTableTuples = alltogethernowResults[0]
        baseScore = alltogethernowResults[1]
    except RuntimeError:
        with open('redos','a') as f:
            bigString = simImage+','+str(fullname)+','+str(fullValList)
            f.write(bigString)
            basescore = -1
        print('Redo with valList = ',fullValList)
        pass
    print(baseScore)
    return [finalTableTuples,baseScore]

In [50]:
# def evaluateImageHillClimbVersionOne(valList,simImage,OGImage,ccd,median,norm,minmag,smallName='newRun',atList=['DETECT_THRESH','DETECT_MINAREA','FILTER_NAME'],starting=[0,0,0]):
    
#     finalTableTuples = []
#     t0 = time.time()
#     scores = []
    
#     for loop in range(2):
#         for x in range(len(valList)):
#             scorePerVal = []
#             for y in range(len(valList[x])):
#                 starting[x] = y
#                 fullname = smallName+'_'+str(starting[0])+'_'+str(starting[1])+'_'+str(starting[2])                
#                 variableList = [float(valList[0][starting[0]]),float(valList[1][starting[1]]),valList[2][starting[2]],ccd,median,minmag,norm]
#                 fullValList = [valList[0][starting[0]],valList[1][starting[1]],valList[2][starting[2]]]
#                 print(starting)
#                 print(fullname)
#                 try:
#                     alltogethernowResults = alltogethernow(fullValList,tableName=fullname,IMAGE=simImage,variableList=variableList,originalImage=OGImage,mag=minmag,atList=atList)
#                     finalTableTuples.append(alltogethernowResults[0])
#                     scorePerVal.append(alltogethernowResults[1])
#                 except RuntimeError:
#                     with open('redos','a') as f:
#                         bigString = simImage+','+str(fullname)+','+str(fullValList)
#                         f.write(bigString)
#                         scorePerVal.append(-1)
#                     print('Redo with valList = ',fullValList)
#                     pass
#             goodScores = [sco for sco in scorePerVal if x >= 0]
#             best = scorePerVal.index(min(goodScores))
#             starting[x] = best

#     print(finalTableTuples)
#     rdd = sc.parallelize(finalTableTuples)
#     kepler = rdd.map(lambda x: Row(detectThresh=x[0],detectMinarea=x[1],filterName=x[2],ccd=x[3],median=x[4],medianQuality=x[5],
#                                    minmag=x[6], xy_0=int(x[7]),xy_1=int(x[8]),xy_2=int(x[9]),xy_3=int(x[10]),xy_4=int(x[12]),
#                                    xy_5=int(x[12]),xy_6=int(x[13]),xy_7=int(x[14]),xy_8=int(x[15]),xy_9=int(x[16]),xy_10=int(x[17]),
#                                    xy_11=int(x[18]),xy_12=int(x[19]),xy_13=int(x[20]),xy_14=int(x[21]),xy_15=int(x[22]),totNum=int(x[23]),totGross=int(x[24]),OGtotGross=int(x[25])))

#     schemaKepler = sqlContext.createDataFrame(kepler)
#     #print('finalResults_{}_{}_{}'.format(ccd,median,minmag))
#     schemaKepler.write.format('jdbc').options(url='jdbc:mysql://localhost/Kepler',driver='com.mysql.jdbc.Driver',dbtable='finalResultsNewRun_{}_{}_{}'.format(ccd,median,abs(minmag)),user='mj1e16',password='[sqlT1G3R]').mode('append').save()
    
#     print(time.time()-t0)
#     return starting


In [76]:
def evaluateImageHillClimbVersionOne(valList,simImage,OGImage,ccd,median,norm,minmag,smallName='newRun',atList=['DETECT_THRESH','DETECT_MINAREA','FILTER_NAME'],starting=[0,0,0]):
    
    finalTableTuples = []
    t0 = time.time()
    scores = []
    
    startingBest = 10**10
    best = 10**9
    
    while startingBest > best:
        startingBest = best
        for x in range(len(valList)):
            scorePerVal = []
            for y in range(len(valList[x])):
                starting[x] = y
                baseEvaluate = oneSettingEvaluate(valList,simImage,OGImage,ccd,median,norm,minmag,smallName=smallName,starting=starting,atList=atList)
                finalTableTuples.append(baseEvaluate[0])
                scorePerVal.append(baseEvaluate[1])

            goodScores = [sco for sco in scorePerVal if x >= 0]
            best = scorePerVal.index(min(goodScores))
            bestList = [i for i, val in enumerate(goodScores) if val == min(goodScores)]
            if len(bestList) == 1:
                best = bestList[0]
            else:
                best = int(np.median(bestList))
            starting[x] = best
            print(scorePerVal)

    print(finalTableTuples)
    
    rdd = sc.parallelize(finalTableTuples)
    kepler = rdd.map(lambda x: Row(detectThresh=x[0],detectMinarea=x[1],filterName=x[2],ccd=x[3],median=x[4],medianQuality=x[5],
                                   minmag=x[6], xy_0=int(x[7]),xy_1=int(x[8]),xy_2=int(x[9]),xy_3=int(x[10]),xy_4=int(x[11]),
                                   xy_5=int(x[12]),xy_6=int(x[13]),xy_7=int(x[14]),xy_8=int(x[15]),xy_9=int(x[16]),xy_10=int(x[17]),
                                   xy_11=int(x[18]),xy_12=int(x[19]),xy_13=int(x[20]),xy_14=int(x[21]),xy_15=int(x[22]),
                                   totNum=int(x[23]),totGross=int(x[24]),OGtot=int(x[25]),OGtotGross=int(x[26]),score=int(x[27])))

    schemaKepler = sqlContext.createDataFrame(kepler)
    #print('finalResults_{}_{}_{}'.format(ccd,median,minmag))
    schemaKepler.write.format('jdbc').options(url='jdbc:mysql://localhost/Kepler',driver='com.mysql.jdbc.Driver',dbtable='finalResultsSimpleLoop_{}_{}_{}'.format(ccd,median,abs(minmag)),user='mj1e16',password='[sqlT1G3R]').mode('append').save()
    
    print(time.time()-t0)
    return starting


In [77]:
def evaluateImageHillClimbVersionTwo(valList,simImage,OGImage,ccd,median,norm,minmag,baseNum,smallName='newRun',starting=[0,0,0],atList=['DETECT_THRESH','DETECT_MINAREA','FILTER_NAME']):
    
    finalTableTuples = []
    t0 = time.time()
    scores = []
    
    baseEvaluate = oneSettingEvaluate(valList,simImage,OGImage,ccd,median,norm,minmag,smallName=smallName,starting=starting,atList=atList)
    finalTableTuples.append(baseEvaluate[0])
    baseScore = baseEvaluate[1]
    startScore = baseScore + 1
    
    plusMinus = [-1,1]
    while baseScore < startScore:
        startScore = baseScore
        for x in range(len(valList)):
            for y in plusMinus:
                delta = -1
                while delta < 0:
                    minus = [z for z in starting]
                    minus[x] += y
                    if 0 < minus[x] < len(valList[x]):
                        minusEvaluate = oneSettingEvaluate(valList,simImage,OGImage,ccd,median,norm,minmag,baseNum,smallName=smallName,starting=minus,atList=atList)
                        finalTableTuples.append(minusEvaluate[0])
                        minusScore = minusEvaluate[1]

                        delta = minusScore - baseScore

                        randomVals = [random.randint(0,len(valList[0])-1),random.randint(0,len(valList[1])-1),random.randint(0,len(valList[2])-1)]
                        randomEvaluate = oneSettingEvaluate(valList,simImage,OGImage,ccd,median,norm,minmag,baseNum,smallName=smallName,starting=randomVals,atList=atList)
                        finalTableTuples.append(randomEvaluate[0])
                        randomScore = randomEvaluate[1]

                        if randomScore < baseScore:
                            starting = randomVals
                            delta = randomScore - baseScore
                            baseScore = randomScore

                        elif minusScore < baseScore:
                            starting = minus
                            baseScore = minusScore
                    else:
                        break
                
                

    print(finalTableTuples)
    print(baseScore)
    print(starting)
    rdd = sc.parallelize(finalTableTuples)
    kepler = rdd.map(lambda x: Row(detectThresh=x[0],detectMinarea=x[1],filterName=x[2],ccd=x[3],median=x[4],medianQuality=x[5],
                                   minmag=x[6], xy_0=int(x[7]),xy_1=int(x[8]),xy_2=int(x[9]),xy_3=int(x[10]),xy_4=int(x[11]),
                                   xy_5=int(x[12]),xy_6=int(x[13]),xy_7=int(x[14]),xy_8=int(x[15]),xy_9=int(x[16]),xy_10=int(x[17]),
                                   xy_11=int(x[18]),xy_12=int(x[19]),xy_13=int(x[20]),xy_14=int(x[21]),xy_15=int(x[22]),
                                   totNum=int(x[23]),totGross=int(x[24]),OGtot=int(x[25]),OGtotGross=int(x[26]),score=int(x[27])))

    schemaKepler = sqlContext.createDataFrame(kepler)
    #print('finalResults_{}_{}_{}'.format(ccd,median,minmag))
    schemaKepler.write.format('jdbc').options(url='jdbc:mysql://localhost/Kepler',driver='com.mysql.jdbc.Driver',dbtable='finalResultsHillClimb_{}_{}_{}'.format(ccd,median,abs(minmag)),user='mj1e16',password='[sqlT1G3R]').mode('append').save()
    
    print(time.time()-t0)
    return starting

In [11]:
#sqlContext.cacheTable('starlist')

In [40]:
#sqlContext.sql('DROP TABLE IF EXISTS finalResults_44_1')
#valList = [[1.],[1.],[8.],[3.]]
#valList = [np.linspace(1,2,2),np.linspace(1,2,2),[8.],[3.]]

In [19]:
# ccd = [44,63,79]
# median = [0,1,2]
# mags = np.linspace(-7,-1,7)

#ccd = [44] #,63,79]
ccd = [44]
median = [1]
mags = [-7,-6,-5,-4,-3,-2,-1]


In [20]:

norms = []
for inc,c in enumerate(ccd):
    totVals = []
    for x in range(3):
        totVals.append(hotpantsQuality('/home/mj1e16/Simages/diff_{}_{}.fits'.format(c,x)))
    norms.append([float(y/totVals[1]) for y in totVals])
print(norms)

[[128.10734802244414, 1.0, 1.207334982887684]]


In [89]:
### Make function to replace starlist
bestSettings = []
baseSettings = [valList[0][0],valList[1][0],valList[2][0]]
starting = [4,4,14]
for inc,c in enumerate(ccd):
    for inmed,med in enumerate(median):
        ogim = '/home/mj1e16/Simages/diff_{}_{}.fits'.format(c,med)
        baseNames = makeConfig(baseSettings,tableName='baseTable')
        baseNum = findObjects(baseNames[0],baseNames[1],baseNames[2],imagename=ogim,base='yes')
        norm = norms[0][1]
        #subprocess.call(['mkdir','/home/mj1e16/keplerPhotometry/provDump/{}/{}'.format(c,med)])
        for mag in mags:
            sqlContext.cacheTable('starlist_{}'.format(abs(mag)))
            #subprocess.call(['mkdir','/data/mj1e16/provDump/{}/{}/{}'.format(c,med,mag)])
            
            imName = '/home/mj1e16/Simages/diff_{}_{}{}_alt.fits'.format(c,med,mag)
            bestSettings.append(evaluateImageHillClimbVersionTwo(valList,imName,ogim,'ccd_{}'.format(c),'median_{}'.format(med),norm,mag,baseNum,smallName='newRun5_{}_{}_{}'.format(c,med,mag),starting=[4,6,12]))
            sqlContext.uncacheTable('starlist_{}'.format(abs(mag)))
            starting = bestSettings[-1]


newRun5_44_1_-7_4_6_12
[4, 6, 12]
148
newRun5_44_1_-7_3_6_12
[3, 6, 12]
146
newRun5_44_1_-7_5_1_18
[5, 1, 18]
146
newRun5_44_1_-7_4_1_18
[4, 1, 18]
144
newRun5_44_1_-7_0_1_13
[0, 1, 13]
141
newRun5_44_1_-7_1_1_13
[1, 1, 13]
138
newRun5_44_1_-7_5_1_12
[5, 1, 12]
145
newRun5_44_1_-7_2_1_13
[2, 1, 13]
138
newRun5_44_1_-7_8_8_5
[8, 8, 5]
140
newRun5_44_1_-7_1_2_13
[1, 2, 13]
138
newRun5_44_1_-7_4_6_11
[4, 6, 11]
160
newRun5_44_1_-7_1_1_12
[1, 1, 12]
143
newRun5_44_1_-7_3_0_17
[3, 0, 17]
140
newRun5_44_1_-7_1_1_14
[1, 1, 14]
129
newRun5_44_1_-7_9_5_18
[9, 5, 18]
153
newRun5_44_1_-7_1_1_15
[1, 1, 15]
146
newRun5_44_1_-7_1_8_4
[1, 8, 4]
137
newRun5_44_1_-7_2_1_14
[2, 1, 14]
129
newRun5_44_1_-7_9_4_15
[9, 4, 15]
144
newRun5_44_1_-7_1_2_14
[1, 2, 14]
130
newRun5_44_1_-7_6_1_11
[6, 1, 11]
144
newRun5_44_1_-7_1_1_13
[1, 1, 13]
138
newRun5_44_1_-7_2_4_16
[2, 4, 16]
142
newRun5_44_1_-7_1_1_15
[1, 1, 15]
146
newRun5_44_1_-7_5_5_18
[5, 5, 18]
146
[(5.0, 7.0, 'mexhat_4.0_9x9.conv', 'ccd_44', 'median_1

122.659696102
newRun5_44_1_-5_4_6_12
[4, 6, 12]
154
newRun5_44_1_-5_3_6_12
[3, 6, 12]
154
newRun5_44_1_-5_6_0_19
[6, 0, 19]
155
newRun5_44_1_-5_5_6_12
[5, 6, 12]
154
newRun5_44_1_-5_2_4_12
[2, 4, 12]
153
newRun5_44_1_-5_3_4_12
[3, 4, 12]
154
newRun5_44_1_-5_5_4_6
[5, 4, 6]
146
newRun5_44_1_-5_6_4_6
[6, 4, 6]
145
newRun5_44_1_-5_9_8_7
[9, 8, 7]
149
newRun5_44_1_-5_7_4_6
[7, 4, 6]
146
newRun5_44_1_-5_5_4_14
[5, 4, 14]
147
newRun5_44_1_-5_6_3_6
[6, 3, 6]
145
newRun5_44_1_-5_3_1_5
[3, 1, 5]
140
newRun5_44_1_-5_3_2_5
[3, 2, 5]
140
newRun5_44_1_-5_4_1_9
[4, 1, 9]
126
newRun5_44_1_-5_4_2_9
[4, 2, 9]
125
newRun5_44_1_-5_8_2_16
[8, 2, 16]
138
newRun5_44_1_-5_4_3_9
[4, 3, 9]
152
newRun5_44_1_-5_8_2_13
[8, 2, 13]
149
newRun5_44_1_-5_4_2_8
[4, 2, 8]
184
newRun5_44_1_-5_8_7_6
[8, 7, 6]
143
newRun5_44_1_-5_4_2_10
[4, 2, 10]
148
newRun5_44_1_-5_5_4_3
[5, 4, 3]
142
newRun5_44_1_-5_3_2_9
[3, 2, 9]
125
newRun5_44_1_-5_8_5_7
[8, 5, 7]
149
newRun5_44_1_-5_5_2_9
[5, 2, 9]
125
newRun5_44_1_-5_3_8_0
[3, 8, 0

137
newRun5_44_1_-4_6_1_2
[6, 1, 2]
140
newRun5_44_1_-4_9_1_15
[9, 1, 15]
141
newRun5_44_1_-4_8_1_11
[8, 1, 11]
145
[(5.0, 7.0, 'mexhat_4.0_9x9.conv', 'ccd_44', 'median_1', -4, 1.0, 67, 58, 66, 49, 68, 64, 68, 50, 64, 48, 59, 30, 69, 58, 46, 47, 890, 1286, 0, 301, 145), (4.0, 7.0, 'mexhat_4.0_9x9.conv', 'ccd_44', 'median_1', -4, 1.0, 67, 59, 66, 49, 68, 64, 68, 50, 64, 48, 59, 30, 69, 58, 46, 47, 891, 1396, 0, 422, 144), (1.0, 9.0, 'tophat_2.5_3x3.conv', 'ccd_44', 'median_1', -4, 1.0, 69, 60, 65, 49, 68, 64, 67, 49, 63, 50, 58, 31, 69, 57, 37, 47, 883, 4482, 5, 4122, 151), (3.0, 7.0, 'mexhat_4.0_9x9.conv', 'ccd_44', 'median_1', -4, 1.0, 67, 60, 66, 49, 68, 64, 68, 50, 64, 48, 59, 30, 69, 58, 46, 47, 892, 1558, 0, 596, 143), (9.0, 3.0, 'gauss_2.0_5x5.conv', 'ccd_44', 'median_1', -4, 1.0, 69, 64, 67, 50, 68, 64, 68, 50, 61, 50, 59, 27, 69, 60, 46, 47, 897, 1449, 0, 557, 143), (8.0, 3.0, 'gauss_2.0_5x5.conv', 'ccd_44', 'median_1', -4, 1.0, 69, 64, 67, 50, 68, 64, 67, 50, 61, 50, 59, 27, 6

89.6754908562
newRun5_44_1_-2_4_6_12
[4, 6, 12]
147
newRun5_44_1_-2_3_6_12
[3, 6, 12]
145
newRun5_44_1_-2_4_3_11
[4, 3, 11]
146
newRun5_44_1_-2_3_3_11
[3, 3, 11]
145
newRun5_44_1_-2_6_4_17
[6, 4, 17]
139
newRun5_44_1_-2_5_4_17
[5, 4, 17]
138
newRun5_44_1_-2_9_2_7
[9, 2, 7]
156
newRun5_44_1_-2_4_4_17
[4, 4, 17]
138
newRun5_44_1_-2_4_6_11
[4, 6, 11]
165
newRun5_44_1_-2_6_4_17
[6, 4, 17]
139
newRun5_44_1_-2_4_5_2
[4, 5, 2]
140
newRun5_44_1_-2_5_3_17
[5, 3, 17]
138
newRun5_44_1_-2_4_0_1
[4, 0, 1]
143
newRun5_44_1_-2_5_5_17
[5, 5, 17]
138
newRun5_44_1_-2_9_8_3
[9, 8, 3]
141
newRun5_44_1_-2_5_4_16
[5, 4, 16]
137
newRun5_44_1_-2_4_1_0
[4, 1, 0]
136
newRun5_44_1_-2_4_1_1
[4, 1, 1]
140
newRun5_44_1_-2_6_4_5
[6, 4, 5]
140
newRun5_44_1_-2_3_1_0
[3, 1, 0]
135
newRun5_44_1_-2_2_5_3
[2, 5, 3]
142
newRun5_44_1_-2_2_1_0
[2, 1, 0]
137
newRun5_44_1_-2_1_8_15
[1, 8, 15]
155
newRun5_44_1_-2_4_1_0
[4, 1, 0]
136
newRun5_44_1_-2_2_1_7
[2, 1, 7]
160
newRun5_44_1_-2_3_2_0
[3, 2, 0]
136
newRun5_44_1_-2_5_2_4
[5

89.1950469017


In [86]:
print(bestSettings)

[[1, 1, 14], [2, 3, 3], [4, 5, 2], [1, 3, 19], [5, 1, 14], [4, 6, 17], [5, 1, 14]]


In [88]:
print(valList[2][14])

tophat_1.5_3x3.conv


In [49]:
a = [1.0, 1.0, 8.0, 3.0, 'ccd_44', 'median_1', -7, 1.0, 67, 62, 64, 48, 70, 67, 68, 51, 70, 67, 64, 30, 70, 67, 68, 48, 67, 62, 64, 48, 70, 67, 68, 51, 70, 67, 64, 30, 70, 67, 68, 48, 967, 15760]

In [32]:
a = [1.0, 1.0, 8.0, 3.0, 'ccd_44', 'median_1', -7, 1.0, 67, 67, 62, 62, 64, 64, 48, 48, 70, 70, 67, 67, 68, 68, 51, 51, 70, 70, 67, 67, 64, 64, 30, 30, 70, 70, 67, 67, 68, 68, 48, 48, 967, 15760]

In [33]:
a[24]

70

In [14]:
tup = [(1,2,3),(4,5,6)]
rdd = sc.parallelize(tup)
kepler = rdd.map(lambda x: Row(one=int(x[0]),two=int(x[1]),three=int(x[2])))
schemaKepler = sqlContext.createDataFrame(kepler)
schemaKepler.write.format('jdbc').options(url='jdbc:mysql://localhost/Kepler',driver='com.mysql.jdbc.Driver',dbtable='testTab',user='mj1e16',password='[sqlT1G3R]').mode('append').save()


In [7]:
a = sqlContext.sql("show tables").show()
print(a)

+--------+----------+-----------+
|database| tableName|isTemporary|
+--------+----------+-----------+
|        |starlist_1|       true|
|        |starlist_2|       true|
|        |starlist_3|       true|
|        |starlist_4|       true|
|        |starlist_5|       true|
|        |starlist_6|       true|
|        |starlist_7|       true|
+--------+----------+-----------+

None


In [15]:
def test():
    row = [('tabName',1,1),('tabName2',1,2)]
    rdd = sc.parallelize(row)
    kepler = rdd.map(lambda x: Row(tabName=x[0], xyRegion=int(x[1]),magRegion=int(x[2])))
    schemaKepler = sqlContext.createDataFrame(kepler)
    df = schemaKepler.toPandas()
    print(df)

In [25]:
def testPartial(value):
    return (value) #,value)

In [26]:
p = ThreadPool(6)
values = [0,1,2,3,4,5]
answer = []
for x in range(5):
    answer.extend(p.map(testPartial,values))
print(answer)

[0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5]


In [26]:
import sys

In [27]:
sys.getsizeof(answer)

336

In [12]:
def findMagRange(catName):
    assoc = Table.read(catName,format='ascii.sextractor')
    df = assoc.to_pandas()
    df2 = df[df['MAG_BEST'] != 99.0000]
    #df3 = df[df['MAG_BEST'] == 99.0000]
    maxVal = df2['MAG_BEST'].max()
    minVal = df2['MAG_BEST'].min()
#     df4 = df2.sort_values('MAG_BEST')
#     return df4
    print(minVal,maxVal)

In [32]:
def findMagRange(catName):
    assoc = Table.read(catName,format='ascii.sextractor')
    df = assoc.to_pandas()
    df2 = df[df['MAG_BEST'] != 99.0000]
    #df3 = df[df['MAG_BEST'] == 99.0000]
#     maxVal = df2['MAG_BEST'].max()
#     minVal = df2['MAG_BEST'].min()
    df4 = df2.sort_values('MAG_BEST')
    return df4
#     print(minVal,maxVal)

In [15]:
for x in range(6):
    print('/home/mj1e16/sextractor/sextractor-master/config/test{}.cat'.format(x))
    findMagRange('/home/mj1e16/sextractor/sextractor-master/config/test{}.cat'.format(x))

/home/mj1e16/sextractor/sextractor-master/config/test0.cat
/home/mj1e16/sextractor/sextractor-master/config/test1.cat
/home/mj1e16/sextractor/sextractor-master/config/test2.cat
/home/mj1e16/sextractor/sextractor-master/config/test3.cat
/home/mj1e16/sextractor/sextractor-master/config/test4.cat
/home/mj1e16/sextractor/sextractor-master/config/test5.cat


In [33]:
x = 2
df = findMagRange('/home/mj1e16/sextractor/sextractor-master/config/test{}.cat'.format(x))

In [35]:
df

,NUMBER,MAG_BEST,X_IMAGE,Y_IMAGE,ELLIPTICITY
3025,3026,13.0256,782.0090,185.0009,0.352
2869,2870,13.0305,1052.8906,184.9989,0.036
13573,13574,13.0327,1052.7808,946.0535,0.125
13344,13345,13.0328,239.7816,946.0737,0.031
6969,6970,13.0329,239.5148,437.8912,0.103
11193,11194,13.0329,239.6400,691.9606,0.054
13666,13667,13.0333,781.5560,946.0166,0.260
6968,6969,13.0335,781.4474,438.1546,0.157
7010,7011,13.0338,510.7693,437.9970,0.031
2859,2860,13.0340,239.8327,185.1199,0.095


In [49]:
magRange = np.linspace(15,30,16)

In [50]:
magRange

array([ 15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,  23.,  24.,  25.,
        26.,  27.,  28.,  29.,  30.])

In [57]:
catName = 'test0.cat'

assoc = Table.read(catName,format='ascii.sextractor')
t0 = time.time()
df = assoc.to_pandas()
#df2 = df[df['MAG_BEST'] != 99.0000]
print(time.time()-t0) #, totGross)


0.00117707252502


In [56]:
t0 = time.time()
df_spark = sqlContext.createDataFrame(df)
print(time.time()-t0) #, totGross)

t0 = time.time()
totGross = df_spark.count()
print(time.time()-t0, totGross)

1.73592019081
(0.06544804573059082, 16094)


In [59]:
!head test0.cat

#   1 NUMBER                 Running object number                                     
#   2 MAG_BEST               Best of MAG_AUTO and MAG_ISOCOR                            [mag]
#   3 X_IMAGE                Object position along x                                    [pixel]
#   4 Y_IMAGE                Object position along y                                    [pixel]
#   5 ELLIPTICITY            1 - B_IMAGE/A_IMAGE                                       
         1  20.7173    744.5268     19.6496    0.438
         2  99.0000   1000.7009     19.5017    0.285
         3  99.0000    133.3164     20.2157    0.325
         4  21.8619    138.0000     20.4564    0.499
         5  21.5303    779.9139     23.0000    0.633


In [27]:
x = 0
vals = [[x],[1],[2]]
val = [[x],[3]]
print(val)
x = 1
val = [[x],[3]]
print(val)
vals.append(val[0])

[[0], [3]]
[[1], [3]]


In [4]:
import numpy as np 
smallName = 'test'

In [5]:
valListminus1 = [1,3,5,7,9,11]
for vals in (valListminus1):
    valList = [np.linspace(1,10,10),np.linspace(1,9,9),np.linspace(8,128,16),[vals]]

    finalTableTuples = []
    #p = ThreadPool(1)
    #t0 = time.time()
    for x0 in range(3): #len(valList[0])):
        for x1 in range(3): #len(valList[1])):
                for x2 in range(3): #len(valList[2])):
                    name = smallName+'_'+str(x0)+'_'+str(x1)+'_'+str(x2)+'_'
                    fullname = [name]*len(valList[3])
                    fullValList = []
                    for x3 in range(len(valList[3])):
                        fullname[x3] += str(x3)
                        print(fullname[x3])
                        fullValList.append([valList[0][x0],valList[1][x1],valList[2][x2],valList[3][x3]])

                        #alltogethernow(fullValList[x4],fullname,imageName,provDir)
                    print(fullValList)
#                     try:
#                         finalTableTuples.extend(p.map(partial(alltogethernow,tableName=fullname,IMAGE=simImage,provDir=provDir),fullValList))

test_0_0_0_0
[[1.0, 1.0, 8.0, 1]]
test_0_0_1_0
[[1.0, 1.0, 16.0, 1]]
test_0_0_2_0
[[1.0, 1.0, 24.0, 1]]
test_0_1_0_0
[[1.0, 2.0, 8.0, 1]]
test_0_1_1_0
[[1.0, 2.0, 16.0, 1]]
test_0_1_2_0
[[1.0, 2.0, 24.0, 1]]
test_0_2_0_0
[[1.0, 3.0, 8.0, 1]]
test_0_2_1_0
[[1.0, 3.0, 16.0, 1]]
test_0_2_2_0
[[1.0, 3.0, 24.0, 1]]
test_1_0_0_0
[[2.0, 1.0, 8.0, 1]]
test_1_0_1_0
[[2.0, 1.0, 16.0, 1]]
test_1_0_2_0
[[2.0, 1.0, 24.0, 1]]
test_1_1_0_0
[[2.0, 2.0, 8.0, 1]]
test_1_1_1_0
[[2.0, 2.0, 16.0, 1]]
test_1_1_2_0
[[2.0, 2.0, 24.0, 1]]
test_1_2_0_0
[[2.0, 3.0, 8.0, 1]]
test_1_2_1_0
[[2.0, 3.0, 16.0, 1]]
test_1_2_2_0
[[2.0, 3.0, 24.0, 1]]
test_2_0_0_0
[[3.0, 1.0, 8.0, 1]]
test_2_0_1_0
[[3.0, 1.0, 16.0, 1]]
test_2_0_2_0
[[3.0, 1.0, 24.0, 1]]
test_2_1_0_0
[[3.0, 2.0, 8.0, 1]]
test_2_1_1_0
[[3.0, 2.0, 16.0, 1]]
test_2_1_2_0
[[3.0, 2.0, 24.0, 1]]
test_2_2_0_0
[[3.0, 3.0, 8.0, 1]]
test_2_2_1_0
[[3.0, 3.0, 16.0, 1]]
test_2_2_2_0
[[3.0, 3.0, 24.0, 1]]
test_0_0_0_0
[[1.0, 1.0, 8.0, 3]]
test_0_0_1_0
[[1.0, 1.0, 16.0,

In [18]:
np.linspace(-7,-1,7)

array([-7., -6., -5., -4., -3., -2., -1.])